In [ ]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.unicode'] = True
matplotlib.rcParams['text.latex.preamble'] = [
       '\\usepackage{CJK}',
       r'\AtBeginDocument{\begin{CJK}{UTF8}{gbsn}}',
       r'\AtEndDocument{\end{CJK}}',
]
import matplotlib.pyplot as pl
import numpy as np
import sys
import os
import struct
#pl.style.use('ggplot')

# 1. neuron activation is probalistic.
# 2. sample learning is proactive.
# 3. unsupervised learning leads to reference learning(*)
# 4. small labeled data works too.
# 5. self-organizing map group
# 6. association(*)

# [NOTICE] to realize (3) and (6), we deal with this:
# (1) A compainied activation grows a strong connection
# (2) A decay ratio for strength of existing connections
# (3) It's not a end-to-end system>>> it's side-by-side
#     Input A and Output B all viewed as input signals
#     Output B connected to a hidden layer of net extending from A


# Proactive Probalistic Reference Net

# neural network is being constructed beyond layer based architecture
# neurons are arranged in a form of organization

def response_replaced(x, w):
    return np.dot(x,w)/(np.sqrt(np.dot(x,x))*np.sqrt(np.dot(w,w)))
def response(x, w):
    return 1.0 - np.max(np.abs(x-w))

def load_mnist(im_path, lb_path):
    # loading images
    binfile = open(im_path, 'rb')
    buf = binfile.read()
    index = 0
    magic,numImages,numRows,numColumns = \
    struct.unpack_from('>IIII' , buf , index)
    index += struct.calcsize('>IIII')
    if magic!=2051:
        raise NameError('MNIST TRAIN-IMAGE INCCORECT!')
    ims = np.zeros([numImages, numRows*numColumns])
    for i in range(numImages):
        ims[i,:] = struct.unpack_from('>784B', buf, index)
        index += struct.calcsize('>784B');
    # loading labels
    binfile = open(lb_path, 'rb')
    buf = binfile.read()
    index = 0
    magic,numLabels = struct.unpack_from(
        '>II', 
        buf, 
        index
    )
    index += struct.calcsize('>II')
    if magic!=2049:
        raise NameError('MNIST TRAIN-LABEL INCORRECT!')
    lbs = np.zeros(numLabels)
    lbs[:] = struct.unpack_from(
        '>'+ str(numLabels) +'B', 
        buf, 
        index
    )
    return [ims, numRows, numColumns, lbs]


class PPRN:
    eta = 0
    in_dim = []
    out_dim = []
    hid_dep = 0
    hid_dim = []
    som_dim = []
    hid = []
    som = []
    def init(self, in_dim, out_dim, hid_dep, eta):
        self.eta = eta
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hid_dep = hid_dep
        self.hid_dim = np.zeros(hid_dep, dtype=np.int)
        self.som_dim = np.zeros([hid_dep, 3], dtype=np.int)
        self.hid = list(range(hid_dep))
        self.som = list(range(hid_dep))
        times = 1
        init_hid_dim = 2**hid_dep
        for i in range(hid_dep): 
            times = times * 2
            self.hid_dim[i] = times * self.in_dim
            self.som_dim[i, :] = [self.hid_dim[i], init_hid_dim/times, self.hid_dim[i]/2]
            self.hid[i] = np.zeros([self.som_dim[i,0], self.som_dim[i,1]])
            self.som[i] = np.random.rand(self.som_dim[i, 0], self.som_dim[i, 1], self.som_dim[i, 2])
        print 'init done.'
    # 1. layer by layer training
    # 2. probalistic activation
    # 3. selective learning for the supervised learning
    def train(self, x, y):
        for i in range(self.hid_dep):
            for j in range(self.som_dim[i,0]):
                for k in range(self.som_dim[i,1]):
                    if i==0:
                        self.hid[i][j,k] = response(x, self.som[i][j,k,:])
                    else:
                        self.hid[i][j,k] = response(self.hid[i-1][:,0], self.som[i][j,k,:])
                m = np.max(self.hid[i][j,:])
                if m > np.random.rand(1): # probalistic activation
                    # update all patterns in som according to its response value
                    if i == 0:
                        for k in range(self.som_dim[i,1]):
                            self.som[i][j,k,:] += \
                            self.eta*(self.hid[i][j,k]/m)*(x-self.som[i][j,k,:])
                    else:
                        for k in range(self.som_dim[i,1]):
                            self.som[i][j,k,:] += self.eta*(self.hid[i][j,k]/m)*\
                            (self.hid[i-1][:,0]-self.som[i][j,k,:])
                    # store the maximum value on first element as hidden activation
                    self.hid[i][j,0] = m
                else:
                    # store the maximum value on first element as hidden activation
                    self.hid[i][j,0] = 0

def main():
    print '======== init ========'
    net = PPRN()
    # reading MINST database
    # load training images
    train_im_path = '../MNIST/train-images-idx3-ubyte'
    train_lb_path = '../MNIST/train-labels-idx1-ubyte'
    test_im_path = '../MNIST/t10k-images-idx3-ubyte'
    test_lb_path = '../MNIST/t10k-labels-idx1-ubyte'
    # prepare data for training and testing
    print '===== load mnist ====='
    [ims_train, h, w, lbs] = load_mnist(train_im_path, train_lb_path)
    train_num = ims_train.shape[0]
    numlbl = 10
    # load test mnist data
    [ims_test, h, w, lbs_test] = load_mnist(test_im_path, test_lb_path)
    test_num = ims_test.shape[0]
    # apply a dataset
    net.init(h*w, numlbl, 3, 0.1)
    for i in range(train_num):
        net.train(ims_train[i,:], [])
    print '=== procedure done ==='
    
main()


======== init ========
===== load mnist =====
init done.
